# 1. 데이터 불러오기
사용하는 데이터는 2가지 형식이다.  
- 1. PDF(`.pdf`)
- 2. PPT(`.pptx`)

## 1-1. PDF reader

PDF 파일을 읽어주는 라이브러리는 크게 세 가지(`PyPDF2, PyMuPDF, PdfPlumber`)가 있다.  
하나를 선택해서 사용하기보단, 길이가 가장 긴 라이브러리를 사용해 텍스트를 추출 후  
데이터 전처리 및 후처리를 거쳐 깔끔한 문장으로 구성된 텍스트 데이터(`.txt`)를 생성한다.

In [1]:
"""
Requirements: pip install pypdf2 pymupdf pdfplumber
"""

import PyPDF2 # PyPDF2
import fitz  # PyMuPDF
import pdfplumber # python-pdfplumber


def read_pdf(file_path):
    pypdf_reader = PyPDF2.PdfReader(file_path)
    fitz_reader = fitz.open(file_path)
    pdfplumber_reader  = pdfplumber.open(file_path)

    # 길이 비교를 위한 리스트
    text_list = []

    # 1) pypdf reader
    text = ''
    for page in pypdf_reader.pages:
        text += page.extract_text()
    text_list.append(text)

    # 2) pymupdf
    text = ''
    for page_num in range(len(fitz_reader)):
        page = fitz_reader.load_page(page_num)
        text += page.get_text()
    text_list.append(text)

    # 3) pdfplumber
    text = ''
    for page in pdfplumber_reader.pages:
        text += page.extract_text()
    text_list.append(text)

    # 길이비교 후 가장 긴 텍스트를 결과값으로 반환
    length_list = [len(text) for text in text_list]
    max_idx = length_list.index(max(length_list))
    
    # 메모리 누수 방지(파일 닫기
    fitz_reader.close()
    pdfplumber_reader.close()

    # print(length_list, max_idx) # 길이를 보고싶을 땐 출력
    return text_list[max_idx]

## 1-2. PPTX reader
`pdf, pptx` 두 가지 파일이 모두 존재하는 경우, 비교를 통해 더 텍스트 추출이 잘 된 것을 사용한다.

In [2]:
"""
Requirements: pip install python-pptx
"""

from pptx import Presentation
import pandas as pd


# extract text from slide
def extract_text_from_slide(slide):
    text = ""
    for shape in slide.shapes:
        if hasattr(shape, "text_frame") and shape.text_frame:
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    text += run.text + " "
    return text.strip()

def read_ppt(file_path):
    text = ''
    prs = Presentation(file_path)
    for slide in prs.slides:
        text += extract_text_from_slide(slide) + '\n'
    return text

## 돌아가는 코드

In [3]:
# 기본경로
import os
# 정리 이후 디렉토리 패스 변경, (기존 directory = "./dataset/")
directory = "./dataset/pdf_pptx/"

files = os.listdir(directory)

# 모든 pdf, pptx 불러오기
pdf_files = [file for file in files if file.endswith('.pdf')]
pptx_files = [file for file in files if file.endswith('pptx')]
pdf_files_name = list(map(lambda x:x[:-4], pdf_files))
ppt_files_name = list(map(lambda x:x[:-5], pptx_files))

# pdf기반으로 살펴보되, pptx가 존재하면 ppt도 읽기
for file_name in pdf_files_name:
    file_path = directory + file_name
    read_text = read_pdf(f'{file_path}.pdf')

    # ppt file의 문서 길이가 더 긴 경우, ppt 읽기
    if file_name in ppt_files_name:
        ppt_text = read_ppt(f'{file_path}.pptx')
        if len(ppt_text) > len(read_text):
            read_text = ppt_text

    # 텍스트(.txt) 문서 저장        
    with open(f"{directory}{file_name}.txt", "w", encoding='utf-8') as f:
        f.write(read_text)

In [4]:
import pandas as pd
df = pd.read_csv('output.csv')

df.head()

,Slide Number,Text,processed_text
0,1,편견 을 걷어내면 인재 가 보입니다 직장 내 장애인 인식개선 교육 장애인 고용,1. 편견 을 걷어내면 인재 가 보입니다\n2. 직장 내 장애인 인식개선 교육\n3...
1,2,"장애인 고용 , 기업에게 어떤 의미가 있을까 함께 일하는 동료 이해하기 장애인 ...","1. 장애인 고용, 기업에게 어떤 의미인가? 함께 일하는 동료 이해하기\n2. 장애..."
2,3,"01 장애인 고용 , 기업에게 어떤 의미가 있을까","1. 장애인 고용, 기업에게 어떤 의미인가요?"
3,4,차별도 우대도 없었던 게임 산업 분위기 속 장애인 고용의 돌파구 2011 년 ...,1. 차별도 우대도 없었던 게임 산업 분위기 속 장애인 고용의 돌파구 2011 년 ...
4,5,"장애인 업무의 한계를 없애다 상시 근로자 100,016 명 / 장애인 ...","1. 장애인 업무의 한계를 없애다\n2. 상시 근로자 100,016 명 / 장애 근..."
